# Clean Data 
---

In [1]:
SAVE = True # Save to file
#------------------------

import pandas as pd
pd.set_option('display.max_columns', 500)

import numpy as np
import sys
import os

from os import mkdir

In [2]:
# Functions

## Clean
def clean(df, file, save = True):

    df.columns = (list(df.iloc[2].values)) # Get column names
    df = df.loc[:, df.columns.notnull()]   # Remove nan columns
    df = df[~((df['Data / Hora'] == 'Data / Hora') &
              (df['Pressão Atmosférica'] == 'Pressão Atmosférica'))] # Remove all headers

    df = df[df.iloc[:,0].str.contains(':', na = False) &
            df.iloc[:,0].str.contains('/', na = False)] # Get data rows only

    df.insert(0, 'Data','')
    df.insert(1, 'Hora','')
    df[['Data', 'Hora']] = df['Data / Hora'].str.split(expand = True)
    #df.drop('Data / Hora', axis = 1, inplace = True) # Split into 2 columns
    
    drop_cols = [4, 6, 7, 10, 12, 14, 15, 17, 20, 22 ]
    #    drop_cols = [3, 5, 6, 9, 11, 12, 14, 16, 19, 2]
    df = df.drop(df.columns[drop_cols],axis=1)

    col_names = ['Data', 'Hora', 'Data / Hora',
                 'UmidadeRelativa', 'PressaoAtmosferica',
                 'Temperatura do Ar', 'TemperaturaInterna',
                 'PontoDeOrvalho', 'SensacaoTermica',
                 'RadiacaoSolar', 'DirecaoDoVento',
                 'VelocidadeDoVento', 'Precipitacao']
    df.columns = col_names   

    df['Local'] = os.path.basename(file).split()[0].split('_')[0]

    if save:
        save_to_file(df, file)
    
    return df

def save_to_file(df, file):
    save_path = file.replace('rawdata', 'cleandata')
    save_path = save_path.replace('.xls', '.csv')
    #print('saving to ', save_path, '\n')
    df.to_csv(save_path, sep = ';', index = False)
    
    
## Concatanate

def concatanate(df_list, name, save = True):
    if df_list:
        df = pd.concat(df_list, axis = 0)
        if save:
            save_concat(df, name)
            return df
        else:
            return df

def save_concat(df, name, path = './cleandata/Info pluviometricas/Concatanated Data/'):
    file = os.path.join(path, name) + '.csv'
    df.to_csv(file, sep = ';', index = False)

In [3]:
path = './rawdata/Info pluviometricas'

files = []
directories = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    directories.extend(d)
    for file in f:
        if '.xls' in file:
            files.append(os.path.join(r, file))
            
len(files) # Should be 90!

90

In [4]:
# Create dir

from os import mkdir

if not os.path.exists("./cleandata"):
    mkdir('./cleandata')

if not os.path.exists("./cleandata/Info pluviometricas"):
    mkdir('./cleandata/Info pluviometricas')
    
if not os.path.exists("./cleandata/Info pluviometricas/Concatanated Data/"):
    mkdir('./cleandata/Info pluviometricas/Concatanated Data/')
    
for directory in directories:
    path_ = os.path.join("./cleandata/Info pluviometricas", directory)
    if not os.path.exists(path_):
        mkdir(path_)

#### Clean

In [5]:
dic = {directory: [] for directory in directories}

#Load cleaned data into dictonary
i = 0
for file in files:
    for d in directories:
        if d in file:
            print(f'{i}/{len(files)}')
            filename = os.path.basename(file)
            df = pd.read_excel(file)
            dic[d].append( clean(df, file, SAVE) )
            i += 1

0/90
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
1/90
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
2/90
3/90
4/90
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
5/90
6/90
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
7/90
8/90
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
9/90
10/90
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
11/90
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
12/90
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
13/90
14/90
15/90
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
16/90
17/90
18/90
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
19/90
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
20/90
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
21/90
22

#### Concatanate

In [6]:
concatanated = {}
if SAVE:
    for d in directories:
        concatanated[d] = concatanate(dic[d], d, SAVE) # Concatanate and save
else:
    for d in directories:
        concatanated[d] = concatanate(dic[d], d, SAVE) # Concatanate and save

#### Merge

In [7]:
keys = list(concatanated.keys())
estacao0 = concatanated[keys[0]].copy( deep = True)
estacao1 = concatanated[keys[1]].copy( deep = True)
estacao2 = concatanated[keys[2]].copy( deep = True)
estacao3 = concatanated[keys[3]].copy( deep = True)
estacao4 = concatanated[keys[4]].copy( deep = True)

In [8]:
estacao4.drop(columns=['Data', 'Hora'], inplace = True)

new_cols = []
for col in estacao4.columns:
    if col != 'Data / Hora':
        col = col + '_4'
    new_cols.append(col)
    
estacao4.columns = new_cols

In [9]:
estacao0.drop(columns=['Data', 'Hora'], inplace = True)
estacao1.drop(columns=['Data', 'Hora'], inplace = True)
merge1 = estacao0.merge(estacao1, on = 'Data / Hora', how = 'outer', suffixes = ('_0', '_1'))

estacao2.drop(columns=['Data', 'Hora'], inplace = True)
estacao3.drop(columns=['Data', 'Hora'], inplace = True)
merge2 = estacao2.merge(estacao3, on = 'Data / Hora', how = 'outer', suffixes = ('_2', '_3'))

merge3 = merge1.merge(merge2, on = 'Data / Hora', how = 'outer')
merged = merge3.merge(estacao4, on = 'Data / Hora', how = 'outer')

merged.insert(0, 'Data','')
merged.insert(1, 'Hora','')
df[['Data', 'Hora']] = merged['Data / Hora'].str.split(expand = True)

if SAVE:
    if not os.path.isdir('./cleandata/Info pluviometricas/Merged Data/'):
        mkdir('./cleandata/Info pluviometricas/Merged Data/')
    
    merged.to_csv('./cleandata/Info pluviometricas/Merged Data/merged.csv',
                  decimal = ',', sep = ';', index = False)